In [44]:
import numpy as np
import cvxpy as cp

# Define decision variable (x_ij represents allocation for city i and band j)
x = cp.Variable((22, 3), nonneg=True)  # Decision variable matrix

# Data: populations, venue costs, ticket prices, attendance rates, and management shares
populations = np.array([274648, 3831000, 979882, 715884, 808988, 755078, 2664000, 510823, 
                        641903, 1303000, 879293, 362656, 2314000, 913175, 303255, 1551000, 
                        403364, 633218, 425115, 320742, 716577, 1388000])

venue_costs = np.array([40000, 150000, 50000, 80000, 130000, 100000, 120000, 70000, 90000, 
                        110000, 60000, 40000, 120000, 60000, 50000, 100000, 80000, 90000, 
                        70000, 60000, 90000, 100000])

ticket_prices = np.array([75, 150, 500])  # Ticket prices for Band 1, Band 2, Band 3
attendance_rates = np.array([0.0005, 0.005, 0.01])  # Attendance rates for Band 1, Band 2, Band 3
management_shares = np.array([0.35, 0.20, 0.10])  # Management shares for Band 1, Band 2, Band 3

# Calculate profit matrix (P_ij for each city i and band j)
profits = np.zeros((22, 3))
for i in range(22):
    for j in range(3):
        revenue = ticket_prices[j] * (populations[i] * attendance_rates[j]) * management_shares[j]
        profits[i, j] = revenue - venue_costs[i]

# Objective function: Maximize total profit
objective = cp.Maximize(cp.sum(cp.multiply(profits, x)))

# Constraints (e.g., limit allocation or resources)
constraints = [
    cp.sum(x, axis=1) <= 1,  # Each city can host at most one band
    cp.sum(x, axis=0) == 7   # Each band preforms in 7 cities
]

for i in range(22):
    for j in range(3):
        constraints.append(x[i][j] >= 0)
        constraints.append(x[i][j] <= 1)

# Define and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI, verbose=False)

# Print results
print("Optimal Objective Value (Profit):", problem.value)
print("Optimal x:")
print(np.transpose(x.value))

print("Shadow price for each city's constraint (host at most one band):", constraints[0].dual_value)
print("Shadow price for each band's performance limit (exactly 7 cities):", constraints[1].dual_value)

for i in range(2, 68):
        print(f"Reduced cost of primal variable {i-1}:", constraints[i].dual_value)

Optimal Objective Value (Profit): 6107843.114375001
Optimal x:
[[1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1.]]
Shadow price for each city's constraint (host at most one band): [    -0.       419667.3       91999.6       10971.000625     -0.
     -0.       274617.3        1640.75         -0.        80467.3
  66911.25       9415.7      222117.3       48646.1          -0.
 127667.3          -0.           -0.           -0.           -0.
    980.09625  103217.3     ]
Shadow price for each band's performance limit (exactly 7 cities): [-81575.023125   4982.7      347941.4     ]
Reduced cost of primal variable 1: -0.0
Reduced cost of primal variable 2: 45179.778125000004
Reduced cost of primal variable 3: -0.0
Reduced cost of primal variable 4: -0.0
Reduced cost of primal variable 5: -0.0
Reduced cost of primal variable 6: -0.0
Reduced 